<font face="B Mitra" size=4>
<div dir=rtl align=center>
<br>
<img src="https://aut.ac.ir/templates/tmpl_modern01/images/logo_fa.png" alt="Amirkabir University Logo" width="100">
<br>
<font size=6>
<b>پروژه دوم داده کاوی (بخش اول)</b>
<br>
<font size=5> استاد درس: دکتر فاطمه شاکری
<br>
<font size=5> طراحان پروژه: مهدی رجالی، مهدی غیاثی، سینا ارزبین
<br>
<font size=4> پاییز ۱۴۰۳
<hr>
</div>
</font>

<font face="B Nazanin">
<div dir=rtl>
<p align="justify">
<font size=5><font color='#eb4034'>
توجه داشته باشید علاوه بر پیاده سازی خواسته شده، توضیحات لازم در محل تعریف شده را بیان کنید.
</p>
</font>
</div>
</font>

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
بارگذاری کتابخانه های مورد نیاز

In [ ]:
import pandas as pd
import numpy as np
import warnings
import category_encoders as ce
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
بارگذاری مجموعه داده

In [2]:
df = pd.read_csv('GDSC Dataset.csv')

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
جداسازی مجموعه داده‌های Test ،Train و Validation

<div dir=rtl>
<font face="B Nazanin" size=4>
با نسبت‌های زیر داده‌ها را به صورت تصادفی به سه دسته تقسیم کنید:

- 70٪ برای داده‌های آموزش
- 10٪ برای داده‌های ارزیابی
- ۲۰٪ برای داده‌های تست
</font>
</p>

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.125, random_state=42)

print("Train set size:", len(df_train))
print("Validation set size:", len(df_val))
print("Test set size:", len(df_test))

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
مدیریت مقادیر گم‌شده

<div dir=rtl>
<font face="B Nazanin" size=4>
توابع مورد نیاز برای مدیریت مقادیر گم‌شده را مطابق با توضیحات پیاده‌سازی کنید.
</font>
</p>

<div dir=rtl>
<font face="B Nazanin" size=4>
یکی از راه‌های مناسب برای پر‌کردن مقادیر از دست‌رفته، استفاده از ویژگی‌های مرتبط است. 
برای مثال، ما برای پر‌کردن مقادیر از دست‌رفته در ستون <code>Cancer Type</code> که مربوط به نوع سرطان است، می‌توانیم
به بافت آن نمونه دقت کرده و حدس بهتری در راستای پرکردن این مقدار از دست‌رفته بزنیم.

این کار، یعنی به کارگیری ماهیت و دامنه داده‌ها در تصمیماتی که در فرایند‌های مختلف یادگیری ماشین می‌گیریم، به <b>تزریق اطلاعات دامنه‌ای</b> (Domain Knowledge Injection)
معروف است.

در ادامه، قسمتی از کد مورد نظر برای شما آورده‌شده است و توضیحاتی در راستای تکمیل آن برای شما آورده‌ایم.
</font>
</p>

In [4]:
train_imputation_data = {}

In [5]:
def get_mode(group, column, calculate_mode, default_val=np.nan):
    if calculate_mode:
        mode_val = group.mode()
        train_imputation_data[f'{group.name}_{column}'] = mode_val[0] if not mode_val.empty else default_val
    
    return train_imputation_data.get(f'{group.name}_{column}', default_val)

<div dir=rtl>
<font face="B Nazanin" size=4>
در قطعه کد زیر، ما برای پر‌کردن مقادیر از دست‌رفته در ستون <code>Cancer Type</code>، ابتدا نمونه‌ها را بر حسب دارو و بافتشان
گروه‌گروه کرده‌ایم، و سپس برای هر مقدار از دسته‌رفته، رایج‌ترین نوع سرطان در آن گروه را قرار داده‌ایم.

در ادامه شما باید مقادیر خالی دیگر ویژگی‌ها را به نحو زیر پر کنید:

۱. مقادیر از دست‌رفتهٔ ویژگی‌های <code>['General tissue description', 'Specific tissue description']</code> را با گروه‌بندی بر حسب دارو و نوع سرطان پر کنید.

۲. مقادیر از دست‌رفتهٔ ویژگی‌های <code>['Biological target', 'Target pathway']</code> را با گروه‌بندی بر حسب دارو با مد پر کنید.

۳. مقادیر از دسته‌رفتهٔ ویژگی‌های <code>['Microsatellite instability Status', 'Screen Medium', 'Growth Properties']</code> را با گروه‌بندی بر حسب دارو و نوع کلی بافت پر کنید.

۴. به طور مشابه مورد قبل، مقادیر از دست‌رفتهٔ ویژگی‌های <code>['CNA', 'Gene Expression', 'Methylation']</code> را پر کنید.

۵. در نهایت، تمامی مقادیر از دست‌رفتهٔ باقی‌مانده در <code>DataFrame</code> را با مقدار <code>'Unknown'</code> پر کنید.

همچنین به الگوی موجود حول ذخیره‌سازی نتایج روی مجموعه دادهٔ آموزشی در این قسمت دقت کنید.
این کار جهت پیش‌گیری از نشت داده‌ها می‌باشد، زیرا ما می‌بایست برای پر‌کردن داده‌های اعتبارسنجی و آزمایشی تنها از مقادیر از پیش‌محاسبه‌شده از مجموعهٔ آموزشی استفاده کنیم.

این الگو در قسمت رمزگذاری (Encoding) نیز مهم خواهد بود.
</font>
</p>

In [ ]:
def impute_missing_values(df: pd.DataFrame, train_set: bool):
    df = df.copy()
    
    # Imputation of 'Cancer Type' column
    df['Cancer Type'] = df.groupby(by=['Drug', 'Specific tissue description'], dropna=False)['Cancer Type'].transform(
                            lambda group: group.fillna(get_mode(group, column='Cancer Type', calculate_mode=train_set))
                        )

    df['Cancer Type'] = df.groupby(by=['Drug', 'General tissue description'], dropna=False)['Cancer Type'].transform(
                            lambda group: group.fillna(get_mode(group, column='Cancer Type', calculate_mode=train_set))
                        )
    
    # FILL HERE

    return df

In [7]:
df_train = impute_missing_values(df=df_train, train_set=True)
df_val = impute_missing_values(df=df_val, train_set=False)
df_test = impute_missing_values(df=df_test, train_set=False)

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
آموزش مدل CatBoost:

<div dir=rtl>
<font face="B Nazanin" size=4>
مدل CatBoost را با مطالعهٔ داک این پکیج و مطالب موجود در اینترنت، آموزش دهید
و پیش‌بینی‌های مدل روی هر یک از مجموعه‌های آموزشی، اعتبارسنجی و آزمایشی را تولید و ذخیره کنید.

همچنین در مورد الگوریتم CatBoost مطالعه کنید و به سوالات زیر حول نحوهٔ کارکرد آن پاسخ دهید:

۱. دلیل کارکرد مستقیم این الگوریتم با داده‌های رسته‌ای، وجود یک رمز‌گذاری داخل در این الگوریتم است. در مورد شیوه‌های رمز‌گذاری داخلی در این الگوریتم که یکی One Hot Encoding و دیگری Ordered Target Encoding است مطالعه کنید. توضیح دهید که مدل چگونه میان رمزگذاری با هر یک از این روش‌ها تصمیم‌گیری می‌کند.

۲. تفاوت Ordered Target Encoding با Target Encoding ساده که در جزوه مطرح شده است چیست؟
این نوع از رمز‌گذاری چه مزیتی دارد؟

۳. یکی از مشکلات Target Encoding این بود که اگر یک ویژگی رابطهٔ یکنواخت ضعیفی با متغیر هدف داشت، اطلاعات این ویژگی در رمز گذاری از بین می‌رفت.
الگوریتم CatBoost چگونه تلاش در حل این مسئله می‌کند؟
</font>
</p>

In [ ]:
features = ...
target = ...

X_train = ...
y_train = ...

# Also for validation and test sets

In [ ]:
train_pool = Pool(...)
val_pool = Pool(...)

catboost_model = CatBoostRegressor(
    ...
    # task_type="GPU", # Uncomment for faster training if a dedicated GPU is available
    # devices='0'
)

catboost_model.fit(
    ...
)

In [ ]:
train_predictions_CB = ...
val_predictions_CB = ...
test_predictions_CB = ...

<font face="B Nazanin">
<div dir=rtl>
<p align="justify">
<font size=4>
<b> <font color='#eb4034'> توضیحات پاسخ: </font></b>  لطفا اینجا بنویسید
</p>
</font>
</div>
</font>
<hr>

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
انکدینگ داده‌ها

<div dir=rtl>
<font face="B Nazanin" size=4>
تابع مورد نیاز برای انکدینگ داده‌ها را مطابق با توضیحات پیاده‌سازی کنید.
</font>
</p>

<div dir=rtl>
<font face="B Nazanin" size=4>
پیش از آموزش مدل Random Forest، می‌بایست ویژگی‌های رسته‌ای را رمزگذاری کنیم.

برای این کار با توجه به مطالب تدریس‌شده، نحوهٔ رمز‌گذاری برای هر ستون را مشخص کرده و آنها را رمز‌گذاری کنید.

برای این کار از پکیج <code>category_encoders</code> استفاده کنید (داک مربوط به این پکیج را مطالعه کنید).
دقت کنید که مشابه بخش پر‌کردن مقادیر از دست‌رفته، می‌بایست رمز‌گذار‌ها روی داده‌های آموزشی تنظیم‌شده و سپس داده‌های اعتبارسنجی و آزمایشی با استفاده از رمزگذار‌های از پیش‌تنظیم‌شده رمز‌گذاری شوند.
برای دیدن چگونگی این کار، توابع <code>fit</code>، <code>fit_transform</code> و <code>transform</code> که برای هر رمز‌گذار دلخواه موجود است را بررسی کنید.

تابعی که در زیر برای شما آورده‌شده است، کاملاً دلخواه است. مقادیر ورودی و بدنهٔ آن را تغییر دهید.
</font>
</p>

In [ ]:
def encode_categorical_features(*args, **kwargs):
    
    ...

In [ ]:
df_train_encoded, encoders = encode_categorical_features(...)

df_val_encoded = encode_categorical_features(...)

df_test_encoded = encode_categorical_features(...)

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
آموزش مدل Random Forest:

<div dir=rtl>
<font face="B Nazanin" size=4>
مدل Random Forest را با داده‌های رمز‌گذاری‌شده آموزش دهید و پیش‌بینی‌های مدل روی هر یک از مجموعه‌های آموزشی، اعتبارسنجی و آزمایشی را تولید و ذخیره کنید.
</font>
</p>

In [ ]:
features = ...
target = ...

X_train_encoded = ...
y_train_encoded = ...

# Also for validation and test sets

In [ ]:
rf_model = RandomForestRegressor(...)
rf_model.fit(...)

In [ ]:
train_predictions_RF = ...
val_predictions_RF = ...
test_predictions_RF = ...

<div dir=rtl>
<h2>
<font face="B Nazanin" color="#0099cc">
ارزیابی مدل‌ها:

<div dir=rtl>
<font face="B Nazanin" size=4>
<code>R2 Score</code> را برای هر مدل و بر روی هر کدام از مجموع داده‌ها محاسبه کنید.
</font>
</p>

In [ ]:
...

<div dir=rtl>
<font face="B Nazanin" size=4>
این معیار نشان دهنده چه چیزی است؟ مقادیر آن در هر بازه چه معنایی دارد؟
</font>
</p>


<font face="B Nazanin">
<div dir=rtl>
<p align="justify">
<font size=4>
<b> <font color='#eb4034'> توضیحات پاسخ: </font></b>  لطفا اینجا بنویسید
</p>
</font>
</div>
</font>
<hr>

<div dir=rtl>
<font face="B Nazanin" size=4>
نمودار Barplot را برای Feature Importance های هر دو مدل رسم کنید. از این نمودار در بخش بعدی استفاده خواهیم کرد.
</font>
</p>

In [ ]:
...